In [2]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic, lonlat
import datetime
from tqdm import tqdm
import re

In [3]:
file_path1 = '/Users/yokij/Desktop/NYPD_Complaint_Data_Current__Year_To_Date.csv'
data1 = pd.read_csv(file_path1, low_memory=False)
file_path2 = '/Users/yokij/Desktop/NYPD_Complaint_Data_Historic.csv'
data2 = pd.read_csv(file_path2, low_memory=False)
data1 = data1.drop(columns=['New Georeferenced Column'])
df = pd.concat([data1, data2])

In [4]:
#Keep records from January 2018 to May 2023
df['CMPLNT_FR_DT'] = pd.to_datetime(df['CMPLNT_FR_DT'], errors='coerce')
df = df.sort_values(by='CMPLNT_FR_DT', ascending=True)
df = df[(df['CMPLNT_FR_DT'] >= pd.Timestamp('2018-01-01')) & (df['CMPLNT_FR_DT'] <= pd.Timestamp('2023-05-31'))]

In [5]:
#delete data if its category is NaN
df = df[df['KY_CD'].notna()] #KY_CD is more general than PD_CD
df = df[df['LAW_CAT_CD'].notna()] #This records three categories(felony,misdemeanor and violation)
df = df[df['Lat_Lon'].notna()]

In [6]:
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
6053750,177774631,23.0,MANHATTAN,2018-01-01,16:00:00,04/03/2018,11:50:00,COMPLETED,(null),(null),...,U,NaN,UNKNOWN,UNKNOWN,E,998751.0,226901.0,40.789463,-73.947634,"(40.7894632995555, -73.9476340039424)"
6058680,178119610,67.0,BROOKLYN,2018-01-01,00:01:00,04/11/2018,14:20:00,COMPLETED,(null),(null),...,(null),NaN,<18,BLACK,F,997843.0,175671.0,40.648851,-73.951017,"(40.6488507469884, -73.951016510623)"
6055205,173133785,43.0,BRONX,2018-01-01,01:58:00,01/01/2018,03:05:00,COMPLETED,(null),(null),...,(null),NaN,<18,WHITE HISPANIC,M,1020219.0,239110.0,40.822912,-73.870041,"(40.8229123084767, -73.8700413043181)"
6054120,178674915,24.0,MANHATTAN,2018-01-01,00:01:00,04/27/2018,12:45:00,COMPLETED,(null),(null),...,U,NaN,UNKNOWN,UNKNOWN,D,993369.0,229307.0,40.796074,-73.967067,"(40.7960743128304, -73.9670667458109)"
6060528,173154147,50.0,BRONX,2018-01-01,19:00:00,01/01/2018,19:45:00,COMPLETED,(null),(null),...,F,NaN,25-44,BLACK HISPANIC,M,1010914.0,260940.0,40.882862,-73.903574,"(40.8828621313214, -73.9035744897024)"


In [7]:
#To be consistent with modzcta's crs
def swap_coordinates(point):
    if isinstance(point, Point):
        return Point(point.y, point.x)
    return point

df['Lat_Lon'] = df['Lat_Lon'].apply(swap_coordinates)

In [8]:
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
6053750,177774631,23.0,MANHATTAN,2018-01-01,16:00:00,04/03/2018,11:50:00,COMPLETED,(null),(null),...,U,NaN,UNKNOWN,UNKNOWN,E,998751.0,226901.0,40.789463,-73.947634,"(40.7894632995555, -73.9476340039424)"
6058680,178119610,67.0,BROOKLYN,2018-01-01,00:01:00,04/11/2018,14:20:00,COMPLETED,(null),(null),...,(null),NaN,<18,BLACK,F,997843.0,175671.0,40.648851,-73.951017,"(40.6488507469884, -73.951016510623)"
6055205,173133785,43.0,BRONX,2018-01-01,01:58:00,01/01/2018,03:05:00,COMPLETED,(null),(null),...,(null),NaN,<18,WHITE HISPANIC,M,1020219.0,239110.0,40.822912,-73.870041,"(40.8229123084767, -73.8700413043181)"
6054120,178674915,24.0,MANHATTAN,2018-01-01,00:01:00,04/27/2018,12:45:00,COMPLETED,(null),(null),...,U,NaN,UNKNOWN,UNKNOWN,D,993369.0,229307.0,40.796074,-73.967067,"(40.7960743128304, -73.9670667458109)"
6060528,173154147,50.0,BRONX,2018-01-01,19:00:00,01/01/2018,19:45:00,COMPLETED,(null),(null),...,F,NaN,25-44,BLACK HISPANIC,M,1010914.0,260940.0,40.882862,-73.903574,"(40.8828621313214, -73.9035744897024)"


In [11]:
file_path = '/Users/yokij/Desktop/ny_new_york_zip_codes_geo.min.json'
zipcode = gpd.read_file(file_path, low_memory=False)

In [12]:
zipcode.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,36,12205,3612205,B5,G6350,S,40906445,243508,+42.7187855,-073.8292399,N,"POLYGON ((-73.87052 42.75123, -73.86861 42.756..."
1,36,12009,3612009,B5,G6350,S,135241924,2168637,+42.6975663,-074.0355422,N,"POLYGON ((-74.10891 42.65300, -74.10889 42.653..."
2,36,14804,3614804,B5,G6350,S,144718714,232123,+42.3172588,-077.8479358,N,"POLYGON ((-77.92747 42.34775, -77.92632 42.347..."
3,36,14836,3614836,B5,G6350,S,77612958,131305,+42.5429182,-077.8781933,N,"MULTIPOLYGON (((-77.95599 42.47433, -77.95600 ..."
4,36,14536,3614536,B5,G6350,S,47193482,425175,+42.5439751,-078.0836709,N,"POLYGON ((-78.05030 42.53850, -78.05024 42.538..."


In [13]:
# Function to convert string coordinates to Point geometry
def create_point_from_string(coord_str):
    match = re.match(r'\(([^,]+), ([^)]+)\)', coord_str)
    if match:
        lat, lon = map(float, match.groups())
        return Point(lon, lat)
    return None

df['geometry'] = df['Lat_Lon'].apply(create_point_from_string)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)
zipcode.set_crs(epsg=4326, inplace=True)
result = gpd.sjoin(gdf, zipcode, how="left", op='within')

/Users/yokij/opt/anaconda3/lib/python3.9/site-packages/geopandas/sindex.py:87: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (idx, item) in enumerate(geometry.iteritems())


In [14]:
result.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
6053750,177774631,23.0,MANHATTAN,2018-01-01,16:00:00,04/03/2018,11:50:00,COMPLETED,(null),(null),...,10029,3610029,B5,G6350,S,2136945.0,0.0,+40.7916981,-073.9438757,N
6058680,178119610,67.0,BROOKLYN,2018-01-01,00:01:00,04/11/2018,14:20:00,COMPLETED,(null),(null),...,11226,3611226,B5,G6350,S,3339497.0,0.0,+40.6464480,-073.9566488,N
6055205,173133785,43.0,BRONX,2018-01-01,01:58:00,01/01/2018,03:05:00,COMPLETED,(null),(null),...,10473,3610473,B5,G6350,S,5645604.0,13868.0,+40.8186904,-073.8584741,N
6054120,178674915,24.0,MANHATTAN,2018-01-01,00:01:00,04/27/2018,12:45:00,COMPLETED,(null),(null),...,10025,3610025,B5,G6350,S,1946717.0,0.0,+40.7986009,-073.9666227,N
6060528,173154147,50.0,BRONX,2018-01-01,19:00:00,01/01/2018,19:45:00,COMPLETED,(null),(null),...,10463,3610463,B5,G6350,S,3745408.0,346374.0,+40.8806784,-073.9065404,N


In [15]:
result.set_index('CMPLNT_FR_DT', inplace=True)

In [17]:
weekly_data = result.groupby('ZCTA5CE10').resample('W').size().reset_index(name='total_count')

In [18]:
weekly_data.head()

,ZCTA5CE10,CMPLNT_FR_DT,total_count
0,10001,2018-01-07,2
1,10001,2018-01-14,2
2,10001,2018-01-21,0
3,10001,2018-01-28,3
4,10001,2018-02-04,3


In [19]:
weekly_data.to_csv('weekly_data.csv', index=False)